# E - extract

The first step in the extract phase of this ETL is to import the needed frameworks to run the script in this jupyter notebook. Here we import pandas, sqlalchemy, numpy, and a config file. The below cell contains all of these imports.

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
from config import username, password

### Extract CSV Files

Here we will load the 5 csv files that are located in the "Resources" folder of this repo. To do this step we will set each files pathway to its' own variable.

In [ ]:
mcd_file = "Resources/mcd_menu.csv"
bk_mcd_file = "Resources/bk_mcd_menu.csv"
starbucks_food_file = "Resources/starbucks_food.csv"
starbucks_drink_file = "Resources/starbucks_drink_menu.csv"
subway_file = "Resources/subway_menu.csv"

Next we will use the pandas ".read_csv" functionality to read each of our csv's into a dataframe. This allows us to prepare for the transform step as we can now see the data of each csv cleanly presented through the power of the jupyter notebook.

In [ ]:
mcd_df = pd.read_csv(mcd_file)
bk_mcd_df = pd.read_csv(bk_mcd_file, delimiter=';')
starbucks_food_df = pd.read_csv(starbucks_food_file)
starbucks_drink_df = pd.read_csv(starbucks_drink_file)
subway_df = pd.read_csv(subway_file)

# T - transform

For each of the 5 data frames created we will first take a quick look using the ".head()" functionality. Now we can assess the current state of the dataframe and see what information needs to be transformed in order to get the data frames to all be congruent to the ERD diagram versions we had envisioned.

This most often included: dropping uneeded rows, renaming rows to be lowercase or fit our naming conventions, adding our "food_class" column and assigning the correct number designator for dessert (1) drink (2) or food (3).

Finally we would display our fully transformed data frame and move onto the next data frame to repeat the process as needed.

### First Data Frame Transformation: Subway_df

In [ ]:
subway_df.head()

In [ ]:
subway_df["Category"].unique()

As we can see with the ".head()" there are a number of columns in this data frame that we do not need. There are also columns that we do need but that do not fit our naming conventions.

The ".unique()" on the column "Category" also reveals a category in subways menu that is called "Extra" by viewing the whole database we were able to determine that everything which was classified under the "Extra" fell most in-line with a dessert classification.

We will grab the columns that we want and copy them to a "transformed" data frame. Then we will rename the columns of our new data frame so that they are aligned with our desired naming conventions.

Next we created a condition in which the category "Extra" would recieve the value of 1 signifying it as a dessert. This was accomplished by setting the variables accordingly and using the numpy functionality "select"

All other rows in this new column "food_class" were given the designation of 3 for a food item.

The index was set to "id" and the transformed data frame can now be displayed in its full glory with one more ".head()"

In [ ]:
# extract columns desired for database
subway_transformed = subway_df[["Category", "Unnamed: 0", "Saturated Fat (g)", "Calories"]].copy()

# rename columns
subway_transformed.rename(columns={"Category": "category", 
                                   "Unnamed: 0": "item",
                                   "Saturated Fat (g)": "saturated_fat",
                                   "Calories": "calories"}, inplace=True)

# add "food_class" column
# recognizing that category == Extra are desserts in the dataset
conditions = [(subway_transformed["category"] == "Extra")]

values = [1]

subway_transformed["food_class"] = np.select(conditions, values)

subway_transformed["food_class"].replace(0,3, inplace=True)

# create "id" column
subway_transformed["id"] = subway_transformed.index

subway_transformed.set_index("id", inplace=True)

# display dataframe
subway_transformed.head()

### Second Data Frame Transformation: mcd_df

This second data frame transformation went extremely similar to the first. The df was viewed using a ".head()" so that we could see which columns and other data would need to be transformed.

In [ ]:
mcd_df.head()

We again grab the name of the columns that fit our needs and save them as a list to a variable called "mcd_cols"

This is copied into a new data frame and the column names are updated.

Things are looking good but we are not there just yet...

In [ ]:
# Create a filtered dataframe from specific columns
mcd_cols = ["Category", "Item", "Saturated Fat", "Calories"]
mcd_transformed= mcd_df[mcd_cols].copy()

# Rename the column headers for consistency
mcd_transformed = mcd_transformed.rename(columns={"Category": "category",
                                                    "Item": "item",
                                                    "Saturated Fat": "saturated_fat",
                                                    "Calories": "calories"})

mcd_transformed.head()

Taking another look at the "category" column we can see all of the different classes and make decisions on which of our three "food_class" numbers should go to each.

In [ ]:
# Find full list of categories
mcd_transformed['category'].unique()

Just like we did for the subway example, we assign each category to a conditions variable, which is a list. Then using numpy we can assign our values for the "food_class" column with the "select" functionality.

Now the McDonalds df is looking pretty good!!

In [ ]:
# manually assign each category to a food_class 
conditions = [(mcd_transformed['category'] == 'Breakfast') | (mcd_transformed['category'] == 'Beef & Pork') \
                  | (mcd_transformed['category'] == 'Chicken & Fish') | (mcd_transformed['category'] == 'Salads') \
                  | (mcd_transformed['category'] == 'Snacks & Sides'),
              (mcd_transformed['category'] == 'Beverages') | (mcd_transformed['category'] == 'Smoothies & Shakes') \
                  | (mcd_transformed['category'] == 'Coffee & Tea'),
              (mcd_transformed['category'] == 'Desserts'), 
             ]

values = [3, 2, 1]

mcd_transformed['food_class'] = np.select(conditions, values)

mcd_transformed.head()

### Third Data Frame Transformation: bk_mcd_df

In [ ]:
bk_mcd_df.head()

When we look at this data frame with the jupyter notebook, we see something interesting compared to the first two examples. This data frame is not just for a single resturant, but for both McDonald's and Burger King combined!

We won't deal with that yet but it will force a new bit of code soon.

For now we grab our columns and rename them just as we had done in the previous two examples.

In [ ]:
# Create a filtered dataframe from specific columns
bk_mcd_cols = ["Chain", "Type", "Item", "Saturated Fat (g)", "Calories"]
bk_mcd_transformed= bk_mcd_df[bk_mcd_cols].copy()

# Rename the column headers
bk_mcd_transformed = bk_mcd_transformed.rename(columns={"Type": "category",
                                                    "Item": "item",
                                                    "Saturated Fat (g)": "saturated_fat",
                                                    "Calories": "calories"})

bk_mcd_transformed.head()

Again, we take a look at the "category" column so that we can properly classify the data into one of our three "food_class" numbers.

In [ ]:
# Find full list of categories
bk_mcd_transformed['category'].unique()

a conditions variable is once again created. Values are again assigned using numpy "select" functionality, and our combo-resturant data frame is looking just like the other two before it...

but that is not what we want...

In [ ]:
# manually assign each category to a food_class 
conditions = [(bk_mcd_transformed['category'] == 'Whopper Sandwiches') | (bk_mcd_transformed['category'] == 'Flame Broiled Burgers') \
                  | (bk_mcd_transformed['category'] == 'Chicken & More') | (bk_mcd_transformed['category'] == 'Salads & Sides') \
                  | (bk_mcd_transformed['category'] == 'King Jr Meals - Entrees') | (bk_mcd_transformed['category'] == 'King Jr Meals - Sides') \
                  | (bk_mcd_transformed['category'] == 'Breakfast') | (bk_mcd_transformed['category'] == 'Additional Options') \
                  | (bk_mcd_transformed['category'] == 'Sandwiches') | (bk_mcd_transformed['category'] == 'French Fries') \
                  | (bk_mcd_transformed['category'] == 'Chicken & Sauce') | (bk_mcd_transformed['category'] == 'Salads') \
                  | (bk_mcd_transformed['category'] == 'Salad Dressings'),
              (bk_mcd_transformed['category'] == 'Beverages') | (bk_mcd_transformed['category'] == 'McCafe Coffees') \
                  | (bk_mcd_transformed['category'] == 'King Jr Meals - Beverages') | (bk_mcd_transformed['category'] == 'Shakes/Smoothies') \
                  | (bk_mcd_transformed['category'] == 'Soft Drinks') | (bk_mcd_transformed['category'] == 'Hot Coffees') \
                  | (bk_mcd_transformed['category'] == 'Iced Coffees') | (bk_mcd_transformed['category'] == 'Frappes') \
                  | (bk_mcd_transformed['category'] == 'McCafe Coffees - Nonfat Milk') | (bk_mcd_transformed['category'] == 'McCafe Coffees - Whole Milk') \
                  | (bk_mcd_transformed['category'] == 'McCafe Frappes') | (bk_mcd_transformed['category'] == 'McCafe Smoothies'),
              (bk_mcd_transformed['category'] == 'Desserts') | (bk_mcd_transformed['category'] =='King Jr Meals - Desserts') \
                  | (bk_mcd_transformed['category'] == 'Desserts/Shakes') | (bk_mcd_transformed['category'] =='King Jr Meals - Desserts'), 
             ]

values = [3, 2, 1]

bk_mcd_transformed['food_class'] = np.select(conditions, values)

bk_mcd_transformed.head()

Since we want each resturant to eventually be loaded into its' own table in pgAdmin, we need to seperate the Burger King and McDonald data into to seperate data frames here.

First we need to remove the "space" from the Burger King entry, and adjust the datatypes

In [ ]:
# remove bad data (namely the  ' -   ' values found in the original csv)
bk_mcd_transformed = bk_mcd_transformed[bk_mcd_transformed['saturated_fat'] != ' -   ']

In [ ]:
# convert , decimal place to . and set to float64 datatype
bk_mcd_transformed['saturated_fat'] = bk_mcd_transformed['saturated_fat'].str.replace(',', '.')
bk_mcd_transformed['saturated_fat'] = bk_mcd_transformed['saturated_fat'].astype('float64')

Now the data frame is ready to be split based on what information is present in the "Chain" column.

In [ ]:
# Split bk and mcd into seperate dataframes
bk_transformed =  bk_mcd_transformed.loc[bk_mcd_transformed['Chain'] == 'Burger King']
mcd_2_join_transformed =  bk_mcd_transformed.loc[bk_mcd_transformed['Chain'] == 'Mc Donalds']
mcd_2_join_transformed.head()

Unfortunately this leaves us now with two seperate McDonald's data frames, one from each csv. We can rectify this by doing a right join of each of these df's so that any new information from our newly created McDonald's df is added and any repeat information is skipped.

This will eventually leave us with just one McDonald's df and one Burger King df.

In [ ]:
mcd_transformed_combined = mcd_transformed.merge(mcd_2_join_transformed, how = "right")
mcd_transformed_combined

All that is left to do is to remove the now uneeded "Chain" column in each of these data frames and set their index to "id"

In [ ]:
# Remove the 'Chain' column from the dataframes
mcd_transformed_combined = mcd_transformed_combined.drop(columns=['Chain'])
bk_transformed = bk_transformed.drop(columns=['Chain'])

In [ ]:
mcd_transformed_combined["id"] = mcd_transformed_combined.index
mcd_transformed_combined.set_index("id", inplace=True)
mcd_transformed_combined.head()

In [ ]:
bk_transformed["id"] = bk_transformed.index
bk_transformed.set_index("id", inplace=True)
bk_transformed.head()

### Fourth Transform: starbucks_food_df and starbucks_drink_df

Nothing that we haven't already seen before is done in either of the starbucks df's.

The food based data frame is loaded first. After viewing it's contents, the columns are selected, copied and renamed accordingly using all of the same method's as before.

In [ ]:
starbucks_food_df.head()

In [ ]:
#starbucks_food_df.dtypes

In [ ]:
# Create a filtered dataframe from specific columns
starbs_food_cols = ["Category", "Name", "Calories", "Saturated Fat(g)"]
starbs_food_transformed= starbucks_food_df[starbs_food_cols].copy()

# Rename the column headers
starbs_food_transformed = starbs_food_transformed.rename(columns={"Category": "category",
                                                                "Name": "item",
                                                                "Saturated Fat(g)": "saturated_fat",
                                                                "Calories": "calories"
                                                                })

# Show transformed db
starbs_food_transformed.head()

This was our favorite data frame because all of the items were in the 3 (food) base "food_class" so all that had to be done was to create that new column and set it equal to 3

In [ ]:
# Add new column for class designator
# Set column value equal to "food_class number 3" designating food for all
starbs_food_transformed["food_class"] = 3

starbs_food_transformed.head()

We were wrong...

The "category" column here wasn't as revealing as our previous data frames, so instead we looked at the item column and all of the unique items available. This revealed which of the items should acutally be updated to be in the 1 (dessert) category.

The update was done using the conditions list and numpy functionality again.

In [ ]:
#starbs_food_transformed["item"].unique()

In [ ]:
# Update individual item to dessert class if needed
# Assign each category to a food_class manually
conditions = [(starbs_food_transformed["item"] == "Birthday Cake Pop") |
              (starbs_food_transformed["item"] == "Blueberry Oat Cake") |
              (starbs_food_transformed["item"] == "Chocolate Cake Pop") |
              (starbs_food_transformed["item"] == "Chocolate Chip Cookie") |
              (starbs_food_transformed["item"] == "Chocolate Chip Cookie Dough Cake Pop") |
              (starbs_food_transformed["item"] == "Classic Coffee Cake") |
              (starbs_food_transformed["item"] == "Confetti Sugar Cookie") |
              (starbs_food_transformed["item"] == "Double Chocolate Chunk Brownie") |
              (starbs_food_transformed["item"] == "Frosted Doughnut Cake Pop")|
              (starbs_food_transformed["item"] == "Gluten-Free Marshmallow Dream Bar") |
              (starbs_food_transformed["item"] == "Iced Lemon Loaf Cake") |
              (starbs_food_transformed["item"] == "Old-Fashioned Glazed Doughnut") |
              (starbs_food_transformed["item"] == "Strawberry Cake Pop")
             ]

# This is the value for a dessert
values = [1]

starbs_food_transformed['food_class'] = np.select(conditions, values)
        
# Show transformed db    
starbs_food_transformed.head()

Here we did replaced anything that was changed to a zero back to a 3

In [ ]:
# Change all 0 values back to 3 for food_class
starbs_food_transformed["food_class"].replace(0, 3, inplace = True)
starbs_food_transformed.head()

The same steps are followed again for the Starbucks "drinks" data frame

In [ ]:
starbucks_drink_df.head()

In [ ]:
# Create a filtered dataframe from specific columns
starbs_drink_cols = ["Category", "Name", "Calories", "Saturated fat(g)"]
starbs_drink_transformed= starbucks_drink_df[starbs_drink_cols].copy()

# Rename the column headers
starbs_drink_transformed = starbs_drink_transformed.rename(columns={"Category": "category",
                                                                    "Name": "item",
                                                                    "Saturated fat(g)": "saturated_fat",
                                                                    "Calories": "calories"
                                                                    })

# Show transformed db
starbs_drink_transformed.head()

This data frame really really was our favorite because this time all of the "food_class" was actually drink and could just recieve a blanked "2" for the column

In [ ]:
# Add new column for class designator
# Set column value equal to "food_class number 2" designating drink for all
starbs_drink_transformed["food_class"] = 2

starbs_drink_transformed.head()

In [ ]:
# See all of the different category listings
#starbs_drink_transformed["category"].unique()

So that we could have one table for Starbucks, we combined these two data frames using an outer join. Because the columns matched perfectly, the outer join worked perfectly and we now had our Starbucks menu data frame

In [ ]:
# Join the dataframes to get a starbucks food and drink df
starbs_menu_df = starbs_food_transformed.merge(starbs_drink_transformed, how = "outer")
starbs_menu_df

The column "saturated_fat" was moved infront of the column "calories" here so that it would be congruent with our other data frames

In [ ]:
# Adjust so saturated_fat column is before calories
mid = starbs_menu_df["saturated_fat"]
starbs_menu_df.drop(labels=["saturated_fat"], axis = 1, inplace = True)
starbs_menu_df.insert(2, "saturated_fat", mid)
starbs_menu_df["id"] = starbs_menu_df.index
starbs_menu_df.set_index("id", inplace=True)
starbs_menu_df

# L - Load

## Create database connection

In this step we will be connecting to the database that was made in pgAdmin before this jupyter notebook was run.

Remeber: before running the following cells, you will have needed to run the "ERD.sql" file in pgAdmin and created a "config.py" file with your username and password. Please be sure to follow our provided readme closely before proceeding.

In [ ]:
# Update Username and Password for pgAdmin
# Also update Database Name to match what you created at the start
connection_string = f"{username}:{password}@localhost:5432/FastFood_db"

# Create the engine
engine = create_engine(f'postgresql://{connection_string}')

To ensure everything was set up and the config.py file is running correctly, see if you get the correct names to return back from the engine that is connected to your pgAdmin database!

In [ ]:
# Confirm tables
# You should see ["McDonalds", "Burger_King", "Starbucks", "Subway", "Food_Classes"]
engine.table_names()

## Load DataFrames into database

Using the pandas function ".to_sql" we can load the data frames we transformed in this jupyter notebook to our connected engine. If all of the steps have been followed up to this point, after running the next four cells, you can switch over to pgAdmin to query your new fully populated tables!!

In [ ]:
# Use "to_sql" function to load all transformed dfs' data into postgres

# Starbucks
starbs_menu_df.to_sql(name='Starbucks', con=engine, if_exists='append', index=True)

In [ ]:
# Subway
subway_transformed.to_sql(name='Subway', con=engine, if_exists='append', index=True)

In [ ]:
# McDonalds
mcd_transformed_combined.to_sql(name='McDonalds', con=engine, if_exists='append', index=True)

In [ ]:
# Burger King
bk_transformed.to_sql(name='Burger_King', con=engine, if_exists='append', index=True)